#### 지금 영상에서는 주차등록수 예측 모델을 다뤄보는 것에 집중을 해볼거구요. 그래서 EDA 과정과 전처리를 그 이후 과정을 다뤄보겠습니다. 그래서  그래프 시각화는 생략하겠습니다. 

In [71]:
# 필요한 라이브러리를 가지고 와봅시다.
import matplotlib.pyplot as plt
import pandas as pd

In [61]:
train = pd.read_csv("../data/parking_demand/train_df_errno.csv") 
test = pd.read_csv("../data/parking_demand/test_df.csv")
sub = pd.read_csv("../data/parking_demand/sample_submission.csv")
age = pd.read_csv("../data/parking_demand/age_gender_info.csv")

# 데이터 모양을 한번 볼까요?
train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1008, 14), (150, 2), (16, 23))

In [62]:
# 데이터 유형을 보기 위해 info를 한번 봅시다. 
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        2896 non-null   object 
 1   총세대수        2896 non-null   int64  
 2   임대건물구분      2896 non-null   object 
 3   지역          2896 non-null   object 
 4   공급유형        2896 non-null   object 
 5   전용면적        2896 non-null   float64
 6   전용면적별세대수    2896 non-null   int64  
 7   공가수         2896 non-null   float64
 8   자격유형        2896 non-null   object 
 9   임대보증금       2327 non-null   object 
 10  임대료         2327 non-null   object 
 11  10분내지하철수    2685 non-null   float64
 12  10분내버스정류장수  2892 non-null   float64
 13  단지내주차면수     2896 non-null   float64
 14  등록차량수       2896 non-null   float64
dtypes: float64(6), int64(2), object(7)
memory usage: 339.5+ KB


In [63]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        1008 non-null   object 
 1   총세대수        1008 non-null   int64  
 2   임대건물구분      1008 non-null   object 
 3   지역          1008 non-null   object 
 4   공급유형        1008 non-null   object 
 5   전용면적        1008 non-null   float64
 6   전용면적별세대수    1008 non-null   int64  
 7   공가수         1008 non-null   float64
 8   자격유형        1006 non-null   object 
 9   임대보증금       828 non-null    object 
 10  임대료         828 non-null    object 
 11  10분내지하철수    970 non-null    float64
 12  10분내버스정류장수  1008 non-null   float64
 13  단지내주차면수     1008 non-null   float64
dtypes: float64(5), int64(2), object(7)
memory usage: 110.4+ KB


### 결측치 확인

In [64]:
train.isna().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수      211
10분내버스정류장수      4
단지내주차면수         0
등록차량수           0
dtype: int64

In [65]:
test.isna().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         180
임대료           180
10분내지하철수       38
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

In [66]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0


In [67]:
train.corr()['등록차량수'] # 등록차량수와 관련된 상관계수를 보겠다.

총세대수          0.333440
전용면적          0.112717
전용면적별세대수      0.250513
공가수           0.118910
10분내지하철수     -0.107308
10분내버스정류장수    0.104135
단지내주차면수       0.861338
등록차량수         1.000000
Name: 등록차량수, dtype: float64

In [73]:
# 총세대수를 기준으로 등록차량수를 예측을 해봅시다
# feature가 하나인 단순선형회귀를 해볼거에요. 

sel = ['총세대수']
X_train = train[sel]
X_test = test[sel]

y_train = train['등록차량수']

### 데이터 모델링1

In [74]:
# LinearRegression에 해당하는 라이브러리를 불로와야죠
from sklearn.linear_model import LinearRegression

In [75]:
# LinearRegression 으로 모델링을 해봅시다. 
model = LinearRegression() #머신러닝
model.fit(X_train, y_train) #훈련시키겠다.
pred = model.predict(X_test) # X_test를 가지고 예측을 해보고, 그 결과를 'pred' 변수에 넣어줌
pred

array([519.66988273, 519.66988273, 519.66988273, ..., 414.75241208,
       414.75241208, 414.75241208])

In [76]:
len(pred) # test 셋에 등록차량수 전체 채워짐을 확인을 했습니다.

1008

In [79]:
# submision 데이타 셋을 보면, 몇개를 제출해야 하냐면, 
len(sub)  
#150개를 제출하라고 되어 있음 #단지코드 150개 

150

In [80]:
# test 셋의 단지코드는 몇개인지 한번 볼까요?
len(test['단지코드'].unique())

#150개를 제출해야 하는데, 147개니까, 3개를 채워야 되는 거죠. 

147

In [81]:
import numpy as np

In [85]:
test['등록차량수'] = pred

# 1008개 등록차량수를 다 채워 준 다음> 단지코드별로 등록차량수 평균을 내주면 되겠죠?
test['코드별차량수평균'] = test.groupby('단지코드')['등록차량수'].transform(np.mean)  
test.head(10)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,코드별차량수평균
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0,519.669883,519.669883
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0,519.669883,519.669883
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0,519.669883,519.669883
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0,519.669883,519.669883
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0,519.669883,519.669883
5,C1072,754,아파트,경기도,국민임대,51.71,51,14.0,H,43497000,296780,0.0,2.0,683.0,519.669883,519.669883
6,C1072,754,아파트,경기도,국민임대,51.96,198,14.0,H,43497000,296780,0.0,2.0,683.0,519.669883,519.669883
7,C1072,754,아파트,경기도,국민임대,51.96,67,14.0,H,43497000,296780,0.0,2.0,683.0,519.669883,519.669883
8,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,H,22830000,189840,0.0,3.0,1216.0,688.891610,688.891610
9,C1128,1354,아파트,경기도,국민임대,39.79,30,9.0,H,22830000,189840,0.0,3.0,1216.0,688.891610,688.891610


In [17]:
#단지코드가 중복되는 게 있을 수 있으니까, 중복되는 거 빼고, 다시 index 정렬한다음에, 
test.drop_duplicates(['단지코드'], keep='first').reset_index()

# test_new 라는 변수에 넣어 줍시다.
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index()

#test_new를 불러온다. 
test_new

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,코드별차량수평균
0,0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0,519.669883,519.669883
1,8,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,H,22830000,189840,0.0,3.0,1216.0,688.891610,688.891610
2,17,C1456,619,아파트,부산광역시,국민임대,33.40,82,18.0,A,19706000,156200,0.0,16.0,547.0,481.594994,481.594994
3,26,C1840,593,아파트,전라북도,국민임대,39.57,253,7.0,A,14418000,108130,0.0,3.0,543.0,474.262053,474.262053
4,30,C1332,1297,아파트,경기도,국민임대,39.99,282,11.0,H,28598000,203050,0.0,2.0,1112.0,672.815546,672.815546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,982,C2456,349,아파트,제주특별자치도,국민임대,26.44,24,17.0,H,6992000,117000,0.0,4.0,270.0,405.445217,405.445217
143,986,C1266,596,아파트,충청북도,국민임대,26.94,164,35.0,H,8084000,149910,0.0,1.0,593.0,475.108161,475.108161
144,991,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,C,-,-,0.0,1.0,40.0,340.858925,340.858925
145,993,C1267,675,아파트,경상남도,국민임대,24.87,28,38.0,H,6882000,104370,0.0,1.0,467.0,497.389022,497.389022


In [18]:
#우리는 제출을 해야 하니까, 다 필요한게 아니라, '단지코드', '코드별차량수평균'만 필요하다. 

sub_df = test_new[ ['단지코드', '코드별차량수평균']]
sub_df.columns = ['code', 'num']   #제출용에는 이름이 'code', 'num' 되어 있으니까, 바꿔주고 
sub_df


# 147데이터 밖에 없으니까 3개를 처리해서 다시해 줘야 함 (아래이어서)

,code,num
0,C1072,519.669883
1,C1128,688.891610
2,C1456,481.594994
3,C1840,474.262053
4,C1332,672.815546
...,...,...
142,C2456,405.445217
143,C1266,475.108161
144,C2152,340.858925
145,C1267,497.389022


In [19]:
# 저장히기
sub_df.to_csv('baseline_0712.csv', index=False)

# ⬆제출1


### 제출용 오류 코드를 제출을 위해 3개 값을 0으로 만들고 추가¶

* 테스트셋에서 평가 제외되는 데이터는 'C2675'(2번 사항에 해당), 'C2335', 'C1327'(3번 사항에 해당) 3개 단지입니다.

In [20]:
# 'code'컬럼에 3개 데이터 더 하기 
# key와 value 형태로 
add_dat = {'code':['C2675', 'C2335', 'C1327'],
           'num':['0', '0', '0']}
add_df = pd.DataFrame(add_dat)
add_df

#concet으로 두 df를 합쳐줌
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df.to_csv('third_rf_0714.csv', index=False)

sub_df 'C2675'(2번 사항에 해당), 'C2335', 'C1327

,code,num
0,C1072,519.67
1,C1128,688.892
2,C1456,481.595
3,C1840,474.262
4,C1332,672.816
...,...,...
145,C1267,497.389
146,C2189,414.752
147,C2675,0
148,C2335,0


# ⬆제출2

### 그 다음 총세대수와 더불어 등록차량수와 상관이 있는 feature들을 찾아보자 

In [21]:
train.corr()['등록차량수'] # 등록차량수와 관련된 상관계수를 보겠다.



총세대수          0.333440
전용면적          0.112717
전용면적별세대수      0.250513
공가수           0.118910
10분내지하철수     -0.107308
10분내버스정류장수    0.104135
단지내주차면수       0.861338
등록차량수         1.000000
Name: 등록차량수, dtype: float64

### 결측치 처리

In [22]:
test.isnull().sum()

#먼저 제일 적은 자격유형을 처리하자!

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         180
임대료           180
10분내지하철수       38
10분내버스정류장수      0
단지내주차면수         0
등록차량수           0
코드별차량수평균        0
dtype: int64

In [23]:
train.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수      211
10분내버스정류장수      4
단지내주차면수         0
등록차량수           0
dtype: int64

### 자격유형 결측치 처리

In [26]:
# 자격유형의 결측치만 확인하고 싶을 때
test.loc[test['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,코드별차량수평균
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0,578.333415,578.333415
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0,634.458621,634.458621


In [27]:
null_grp = test.groupby(["단지코드"])
grp1 = null_grp.get_group(('C2411'))
grp2 = null_grp.get_group(('C2253'))

grp1

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,코드별차량수평균
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000,100720,0.0,2.0,840.0,578.333415,578.333415
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000,100720,0.0,2.0,840.0,578.333415,578.333415
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000,100720,0.0,2.0,840.0,578.333415,578.333415
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0,578.333415,578.333415
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000,171480,0.0,2.0,840.0,578.333415,578.333415


In [28]:
grp2

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,코드별차량수평균
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0,634.458621,634.458621
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000,83020,0.0,2.0,173.0,634.458621,634.458621
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000,83020,0.0,2.0,173.0,634.458621,634.458621
261,C2253,1161,상가,강원도,임대상가,13.77,1,0.0,D,NaN,NaN,0.0,2.0,173.0,634.458621,634.458621
262,C2253,1161,상가,강원도,임대상가,22.89,1,0.0,D,NaN,NaN,0.0,2.0,173.0,634.458621,634.458621
263,C2253,1161,상가,강원도,임대상가,22.91,1,0.0,D,NaN,NaN,0.0,2.0,173.0,634.458621,634.458621
264,C2253,1161,상가,강원도,임대상가,23.79,1,0.0,D,NaN,NaN,0.0,2.0,173.0,634.458621,634.458621
265,C2253,1161,상가,강원도,임대상가,23.79,1,0.0,D,NaN,NaN,0.0,2.0,173.0,634.458621,634.458621
266,C2253,1161,상가,강원도,임대상가,23.86,1,0.0,D,NaN,NaN,0.0,2.0,173.0,634.458621,634.458621
267,C2253,1161,상가,강원도,임대상가,23.86,1,0.0,D,NaN,NaN,0.0,2.0,173.0,634.458621,634.458621


In [29]:
# 자격유형 결측치 처리
test.loc[196, "자격유형"] = "A"
test.loc[258, "자격유형"] = "C"
test[test["자격유형"].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,코드별차량수평균


In [30]:
# 자격유형 결측치 처리 확인
test.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         180
임대료           180
10분내지하철수       38
10분내버스정류장수      0
단지내주차면수         0
등록차량수           0
코드별차량수평균        0
dtype: int64

### 10분내버스정류장수 결측치 처리

In [31]:
# 특정 컬럼의 결측치 데이터만 가져오기
train.loc[train['10분내버스정류장수'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2293,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0,1214.0
2294,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2295,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2296,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0


In [32]:
# grouped = train.groupby(['단지코드', '지역'])
# group1 = grouped.get_group( ('N2431', '경상남도')  )
# group1

#방법1
grouped = train.groupby(['단지코드'])
group1 = grouped.get_group( ('N2431')  )
group1

#방법2  (방법1=방법2)
# train[train['단지코드']=='N2431']

# '10분내버스정류장수'의 결측치(NaN)가 4개인줄 알았는데, grouped해서 '단지코드'를 살펴 보니까 'N2431'의 버스정류장수는 '2'임을 확인!!

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2293,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0,1214.0
2294,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2295,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2296,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2350,N2431,1047,아파트,경상남도,국민임대,36.77,272,16.0,A,11217000,233330,0.0,2.0,1066.0,1214.0
2351,N2431,1047,아파트,경상남도,국민임대,46.78,200,16.0,A,24389000,303220,0.0,2.0,1066.0,1214.0


In [33]:
# 데이터 확인 후, MaN값 2로 넣어주기 

#방법1
train['10분내버스정류장수']=train['10분내버스정류장수'].fillna(2.0)
group1

#방법2
# train.loc["10분내버스정류장수"] = 2



# # grouped_2 = train.groupby(['임대건물구분', '지역','공급유형', '자격유형'])
# # group2 = grouped_2.get_group( ('아파트', '경상남도', '공공임대(10년)', 'A')  )
# # group2
# # # 데이터 확인 후, 임의 처리 4
# # train.loc[ train['10분내버스정류장수'].isnull(), "10분내버스정류장수"] = 5
# # train.loc[ train['10분내버스정류장수'].isnull(), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
2293,N2431,1047,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,NaN,NaN,1066.0,1214.0
2294,N2431,1047,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2295,N2431,1047,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2296,N2431,1047,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,NaN,NaN,1066.0,1214.0
2350,N2431,1047,아파트,경상남도,국민임대,36.77,272,16.0,A,11217000,233330,0.0,2.0,1066.0,1214.0
2351,N2431,1047,아파트,경상남도,국민임대,46.78,200,16.0,A,24389000,303220,0.0,2.0,1066.0,1214.0


###  라벨인코딩-자격유형에 대해서
라벨인코딩이라는 것은 이름으로 되어있는 것을 숫자로 혹은 그 반대로 변환하는 것입니다.

In [34]:
print(train.자격유형.unique())
print(test.자격유형.unique())

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O']
['H' 'A' 'E' 'C' 'D' 'G' 'I' 'J' 'K' 'L' 'M' 'N']


In [35]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5,
            'F':6, 'G':7, 'H':8, 'I':9, 'J':10,
            'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }

train['자격유형'] =train['자격유형'].map(mapping).astype(int)
test['자격유형'] =test['자격유형'].map(mapping).astype(int)

train.head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,205.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,205.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,205.0


### 데이터 모델링2

In [36]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [37]:
# 다수선형회귀
sel = ['총세대수', '전용면적', '전용면적별세대수',
       '공가수',  '단지내주차면수', '자격유형']
X = train[sel]

y = train['등록차량수']
X_test = test[sel]

In [38]:
# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=0)

In [39]:
# LinearRegression 으로 모델링 해보기
model = LinearRegression() #머신러닝
model.fit(X_train, y_train) #훈련시키겠다.
pred = model.predict(X_test) #[sel]를 가지고 y_test를 예측해봐 # X_test를 가지고 예측을 해보고, 그 결과를 'pred' 변수에 넣어줌
pred

print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수


학습(score) : 0.7848278438379498
테스트(score) : 0.7841937560157191


In [40]:
# mae
mae = np.mean( np.abs(y_test - pred)  )
print(mae)

# mse
mse_val = np.mean( (y_test - pred)**2  )
print(mse_val)

147.8977567932893
43640.968133126094


In [41]:
# RandomForestRegressor 으로 모델링 해보기

model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9970998857092722
테스트(score) : 0.9861975500031855


In [42]:
mae_val = np.mean( abs( y_test - pred ) )
print( mae_val )
mse_val = np.mean( (y_test - pred) **2 )
print( mae_val )

25.261426927502864
25.261426927502864


In [46]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)
pred[0:10]

array([2315.  ,  407.54,  737.56,  593.77,  107.86,  707.13,  558.78,
        305.27,  392.05,  492.32])

In [59]:
test['등록차량수'] = pred
test['코드별차량수평균'] = test.groupby("단지코드")['등록차량수'].transform(np.mean) # 등록차량수를 다 채워 준 다음> 등록차량수 평균


ValueError: Length of values (869) does not match length of index (1008)

In [58]:
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,코드별차량수평균
0,0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,8,22830000,189840,0.0,2.0,683.0,519.669883,519.669883
1,8,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,8,22830000,189840,0.0,3.0,1216.0,688.891610,688.891610
2,17,C1456,619,아파트,부산광역시,국민임대,33.40,82,18.0,1,19706000,156200,0.0,16.0,547.0,481.594994,481.594994
3,26,C1840,593,아파트,전라북도,국민임대,39.57,253,7.0,1,14418000,108130,0.0,3.0,543.0,474.262053,474.262053
4,30,C1332,1297,아파트,경기도,국민임대,39.99,282,11.0,8,28598000,203050,0.0,2.0,1112.0,672.815546,672.815546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,982,C2456,349,아파트,제주특별자치도,국민임대,26.44,24,17.0,8,6992000,117000,0.0,4.0,270.0,405.445217,405.445217
143,986,C1266,596,아파트,충청북도,국민임대,26.94,164,35.0,8,8084000,149910,0.0,1.0,593.0,475.108161,475.108161
144,991,C2152,120,아파트,강원도,영구임대,24.83,66,9.0,3,-,-,0.0,1.0,40.0,340.858925,340.858925
145,993,C1267,675,아파트,경상남도,국민임대,24.87,28,38.0,8,6882000,104370,0.0,1.0,467.0,497.389022,497.389022


In [48]:
# 대회 안내에 빠져 있다고 한 부분 
add_dat = {'code':['C2675', 'C2335', 'C1327'],
           'num':['0', '0', '0']}
add_df = pd.DataFrame(add_dat)
add_df

,code,num
0,C2675,0
1,C2335,0
2,C1327,0


In [49]:
sub_df = test_new[ ['단지코드', '코드별차량수평균']]
sub_df.columns = ['code', 'num'] # 컬럼이름을  submission 틀에 맞춰 바꿔주고.
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df

,code,num
0,C1072,519.67
1,C1128,688.892
2,C1456,481.595
3,C1840,474.262
4,C1332,672.816
...,...,...
145,C1267,497.389
146,C2189,414.752
147,C2675,0
148,C2335,0


In [50]:
# sub_df.to_csv('third_rf_0714.csv', index=False)
# sub_df.head()

# ⬆제출3